In [1]:
import pyvista as pv
import numpy as np
import math  
import trame
import networkx as nx

from tqdm import tqdm
from Voxel_model import Voxels
from Path_create import Create_Path
from creation_vector import Vector_Creation
from mesh_tools import centrer_0xy , rotation_mesh , midle_xy

# import trimesh
# import pyvox
# import pyvoxsurf

In [ ]:
path = Create_Path('Object\\UM3E_Copy of Cube Extruder (3).gcode')
path.read_gcode(12)

collision = Create_Path('Object\\UM3E_collision.gcode')
collision.read_gcode(11)

path.centrer(collision.tool_path)

mesh_collision = centrer_0xy('Object\collision.stl')

path.figure(mesh_collision)

# Free Path

In [58]:
scale_voxel = 0.5
scale_print= 340
vox = Voxels(scale_print,scale_voxel)

MemoryError: Unable to allocate 2.34 GiB for an array with shape (680, 680, 680) and data type float64

In [24]:
voxels_collision = pv.voxelize(mesh_collision, density=scale_voxel, check_surface=False)
voxels_collision.points = np.divide(voxels_collision.points,scale_voxel).astype(int)
mesh_collision.points = np.divide(mesh_collision.points,scale_voxel).astype(int)

In [53]:
centre = midle_xy(voxels_collision)
voxels_collision.points = vox.translation(voxels_collision.points,centre,vox.midle)
mesh_collision.points = vox.translation(mesh_collision.points,centre,vox.midle)

path.correction(vox.scale_voxel)
path.translat(centre,vox.midle)
path.figure(mesh_collision)

c:\Users\sdesg\Documents\Voxel_py310\myenv\lib\site-packages\pyvista\utilities\helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value="<iframe src='http://localhost:49573/index.html?ui=P_0x18539f59e70_12&reconnect=auto' style='widt…

In [26]:
mesh_head_print=  pv.read('Object\Deposition Head.stl')
voxels_head_print = pv.voxelize(mesh_head_print, density=scale_voxel, check_surface=False)
voxels_head_print.points = np.divide(voxels_head_print.points,vox.scale_voxel).astype(int)
mesh_head_print.points = np.divide(mesh_head_print.points,vox.scale_voxel).astype(int)

In [ ]:
voxels_head_print = voxels_head_print.extract_surface()
voxels_head_print.points[:,2]+=6

In [34]:
vox.add_density(voxels_collision.points)

In [49]:
theta , theta_max = 20 , 80
Circle=Vector_Creation(np.deg2rad(theta),np.deg2rad(theta_max))

position_head_layer_circle = {i: [] for i in Circle }
G = nx.DiGraph()
tool_path = path.tool_path[5000:5100]
num_layers = len(tool_path)
layer_vertex = {i: [] for i in range(num_layers)}

In [51]:

free_path= free_path(vox, tool_path, voxels_head_print, theta , position_head_layer_circle,Circle, G, layer_vertex )

  4%|▍         | 4/100 [00:27<10:53,  6.81s/it]


IndexError: index 680 is out of bounds for axis 0 with size 680

In [50]:
def free_path(vox, tool_path, surface, theta, position_head_layer_circle,Circle,G, layer_vertex ):
        #   build_graph
        source = 's'
        target = 't'
        
        G.add_node(source)
        G.add_node(target)
        
        for i in tqdm(range(num_layers)):
            if i > 0:
                prev_layer = i - 1    # Récupérez les sommets de l'étage précédent
                prev_layer_vertices = layer_vertex[prev_layer]

            for j in Circle:
                for ind, delta in enumerate(np.rad2deg(Circle[j])):
                    
                    if i == 0 :
                        position_rotate = rotation_mesh(surface, delta,j*theta) 
                        position_rotate = np.floor(position_rotate.points).astype(int)
                        position_head_layer_circle[j].append(position_rotate)
                        position = vox.translation(position_rotate,np.array([0,0,0]),tool_path[i])
                    else :
                        position_rotate = position_head_layer_circle[j][ind]
                        position = vox.translation(position_rotate,tool_path[i-1],tool_path[i])
                    
                    if not vox.density(position): # s'il n'y a pas de desnité (pas d'obstacle)
                        current_vertex = (i, j , ind)  #  name of the vertex
                        G.add_node(current_vertex, angle_z=j*theta) # creer le sommet white atribute 
                        layer_vertex[i].append(current_vertex)  # Ajoute les sommets aux listes d'étages
                        if i > 0:
                            for prev_vertex in prev_layer_vertices: # creer des liaison entre les etages si la condition angulaire est respécté
                                angle = abs(G.nodes[prev_vertex]['angle_z']-G.nodes[current_vertex]['angle_z'])
                                if angle <= theta:
                                    G.add_edge(prev_vertex, current_vertex, weight=angle)
                        if i == 0:
                            G.add_edge(source, current_vertex, weight=0)
                        if i == num_layers-1:
                            G.add_edge(current_vertex,target ,weight=0)
                            
            vox.voxels[tool_path[i][0],tool_path[i][1],tool_path[i][2]]=1


        shortest_path = nx.dijkstra_path(G, source, target)
        return shortest_path
    

    # elif methode == 2 :
        """
        Dans cette methode pour chaque vecteur du cercle
        ->je vais creer une un model voxel de le tete (rotation en mesh) -> plus précis
        ->chaque model va etre testé pour chacune des position avec des translation
        """

    # elif methode == 3 :
        """
        ->je vais creer une un model voxel de le tete 
        ->pour chaque vecteur je vais effectuer une rotation (rotation en voxel) -> perte de précision
        ->pour chaque vecteur va etre testé pour chacune des position avec des translation
        """
            # if methode == 1 :  
        """ 
        il faut que j'ajoute le vecteur ou s'effectue la rotation 
        Dans cette methode pour chaque points 
        je vais modifier la position de la tete pour chaque vecteur 
        et determiner si elle est valide ou non
        """


    

In [18]:
free_path=free_path[1:-1]

In [50]:
for ele in free_path:
    G.nodes[ele]

{'vector_xy': 94.03866030650033, 'angle_z': 40}

# Create video

In [19]:
import imageio

def create_images(free_path):
    p = pv.Plotter(off_screen=True)
    images = []

    for ele in tqdm(free_path):
        p.clear()
        position_rotate = rotation_mesh(surface, G.nodes[ele]['vector_xy'],  G.nodes[ele]['angle_z'])
        position_rotate = position_rotate.translate(tool_path[ele[0] + 5000])
        p.add_mesh(position_rotate, color='blue')
        p.add_mesh(surface_collision, color=True, show_edges=True, opacity=0.7)
        img = p.show(screenshot=True, window_size=[800, 600], return_img=True , auto_close=True)  
        images.append(img)

    return images


In [ ]:
free_path_images = create_images(free_path)

In [21]:

def create_video(images, output_file, fps=30):
    with imageio.get_writer(output_file, mode='I', fps=fps) as writer:
        for img in images:
            writer.append_data(img)


create_video(free_path_images, "animation.mp4")



In [ ]:
mesh = pv.read('Object\Extruder.stl')
mesh_scale = mesh.scale([0.25,0.25,0.25], inplace=False)